In [33]:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            import pandas as pd
import numpy as np
import json
import csv
import ijson
import codecs

In [34]:
filename = "rows.json"
with open(filename, 'r') as f:
    objects = ijson.items(f, 'meta.view.columns.item')
    columns = list(objects)

In [35]:
print(columns[0])

{u'name': u'sid', u'format': {}, u'dataTypeName': u'meta_data', u'fieldName': u':sid', u'renderTypeName': u'meta_data', u'position': 0, u'id': -1, u'flags': [u'hidden']}


In [ ]:
column_names = [col["fieldName"] for col in columns]
column_names

[u':sid',
 u':id',
 u':position',
 u':created_at',
 u':created_meta',
 u':updated_at',
 u':updated_meta',
 u':meta',
 u'date_of_stop',
 u'time_of_stop',
 u'agency',
 u'subagency',
 u'description',
 u'location',
 u'latitude',
 u'longitude',
 u'accident',
 u'belts',
 u'personal_injury',
 u'property_damage',
 u'fatal',
 u'commercial_license',
 u'hazmat',
 u'commercial_vehicle',
 u'alcohol',
 u'work_zone',
 u'state',
 u'vehicle_type',
 u'year',
 u'make',
 u'model',
 u'color',
 u'violation_type',
 u'charge',
 u'article',
 u'contributed_to_accident',
 u'race',
 u'gender',
 u'driver_city',
 u'driver_state',
 u'dl_state',
 u'arrest_type',
 u'geolocation']

In [ ]:
good_columns = [
    "date_of_stop", 
    "time_of_stop", 
    "agency", 
    "subagency",
    "description",
    "location", 
    "latitude", 
    "longitude", 
    "vehicle_type", 
    "year", 
    "make", 
    "model", 
    "color", 
    "violation_type",
    "race", 
    "gender", 
    "driver_state", 
    "driver_city", 
    "dl_state",
    "arrest_type"
]
data = []
with open(filename, 'r') as f:
    objects = ijson.items(f, 'data.item')
    for row in objects:
        selected_row = []
        for item in good_columns:
            selected_row.append(row[column_names.index(item)])
        data.append(selected_row)

In [ ]:
df["date_of_stop"]=pd.to_datetime(df["date_of_stop"])

In [ ]:
df = pd.DataFrame(data, columns=good_columns)

In [ ]:
df.to_csv("rows.csv",sep=",", encoding='UTF-8')

In [ ]:
ds = pd.read_csv("rows.csv")

In [ ]:
from pymongo import MongoClient
from datetime import datetime
client = MongoClient()
client = MongoClient('localhost', 27017)
db = client['Assignment']
posts = db.collections
posts.insert_many(df.to_dict('record'))

In [ ]:
 client.close()

In [ ]:
client = MongoClient('localhost', 27017)

In [ ]:
db = client['Assignment']
collections = db.collections

In [ ]:
#Q5.  how many stops are made by car color
car_stops = collections.aggregate([{"$group" : {"_id":"$color", "count":{"$sum":1}}}])
for No in car_stops:
    print No

In [ ]:
# Q6 Find the value counts of arrest type
arrest_type =collections.aggregate([{"$group" : {"_id":"$arrest_type", "count":{"$sum":1}}}])
for No in arrest_type:
    print No

In [ ]:
df["date_of_stop"]

In [ ]:
# Q7  Find which days result in the most traffic stop
Days=collections.aggregate([{"$project":{"No_of_daysRes": { "$dayOfWeek": "$date_of_stop" },} },{
                                 "$group":  {"_id":"$dayRes", "count":{"$sum":1}}}])

for doc in Days:
    print doc